In [ ]:
import numpy as np
import scqubits as scq
import plotly.graph_objects as go

In [ ]:
EJ = 5.5
EC = 0.8
EL = 0.4
params = (EJ, EC, EL)

flxs = np.linspace(0.0, 1.0, 300)

In [ ]:
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=40, truncated_dim=6)
spectrumData = fluxonium.get_matelements_vs_paramvals(
    operator="n_operator", param_name="flux", param_vals=flxs, evals_count=10
)

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(5) if j > i]

energies = spectrumData.energy_table

fig = go.Figure()
for i, j in show_idxs:
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=energies[:, j] - energies[:, i],
            mode="lines",
            name=f"{i} - {j}",
            line=dict(width=2),
        )
    )
fig.add_hline(y=7.2, line_color="black", line_width=2, line_dash="dash")
fig.add_hline(y=5.9, line_color="black", line_width=2, line_dash="dash")
fig.update_yaxes(range=[0.0, 8.0])

fig.show()

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(3) if j > i]

matrixelements = spectrumData.matrixelem_table

fig = go.Figure()
for i, j in show_idxs:
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=np.abs(matrixelements[:, i, j]),
            mode="lines",
            name=f"{i}-{j}",
            line=dict(width=2),
        )
    )
fig.update_layout(
    title=f"EJ/EC/EL = {EJ:.3f}/{EC:.3f}/{EL:.3f}",
    title_x=0.5,
    xaxis_title=r"$\phi_{ext}/\phi_0$",
    yaxis_title="Matrix elements",
)

fig.show()

In [ ]:
Temp = 54.2e-3
Q_cap = 1.0e5
Q_ind = 7.0e7
x_qp = 1.0e-5

scq.settings.T1_DEFAULT_WARNING = False
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=30, truncated_dim=5)
T1_eff_fig, T1_eff_ax = fluxonium.plot_t1_effective_vs_paramvals(
    param_name="flux",
    param_vals=flxs,
    xlim=([flxs.min(), flxs.max()]),
    common_noise_options=dict(i=1, j=0, T=Temp),
    noise_channels=[
        ("t1_capacitive", dict(Q_cap=Q_cap)),
        ("t1_inductive", dict(Q_ind=Q_ind)),
        ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
    ],
)
T1_eff_ax.set_ylim(None, 1e6)